In [27]:
import pandas as pd
import numpy as np
import folium
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors

from pandas.io.json import json_normalize
from urllib.request import urlopen as ureq
from bs4 import BeautifulSoup as soup
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim

In [228]:
CLIENT_ID = '2ZXTFVS1I5KFCDZOZ1FLPR3ZXBMWTJXTXQHN5GXSUSSMIRIU'
CLIENT_SECRET = 'QMP2MWXOZGAMN0XNUJNSBC5YAA4HIXHT0ZY4APUNY5V3VKCF'
ACCESS_TOKEN = 'ZVJZITAP01TU1REJ2TYESFD40W33B3Y4V3QVXCCVTJHN1GXL'
VERSION = '20190630'
LIMIT = 100

In [218]:
address = '8 Lansdowne Road, Bournemouth, BH1 1SD'
geolocator = Nominatim(user_agent='foursquare_agent')
location = geolocator.geocode(address)
latitude = location.latitude

latitude = 50.72041
longitude = -1.87747

In [229]:
search_query = 'office'

miles = 1
radius = miles * 1609.344 # in meters

In [230]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
# url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, ACCESS_TOKEN, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=2ZXTFVS1I5KFCDZOZ1FLPR3ZXBMWTJXTXQHN5GXSUSSMIRIU&client_secret=QMP2MWXOZGAMN0XNUJNSBC5YAA4HIXHT0ZY4APUNY5V3VKCF&ll=50.72041,-1.87747&oauth_token=ZVJZITAP01TU1REJ2TYESFD40W33B3Y4V3QVXCCVTJHN1GXL&v=20190630&query=office&radius=1609.344&limit=100'

In [231]:
results = requests.get(url).json()
print(f"Total number of venues found: {len(results['response']['venues'])}")

Total number of venues found: 23


In [232]:
df = json_normalize(results['response']['venues'])
df.head()

,id,name,categories,referralId,hasPerk,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.city,location.state,location.country,location.formattedAddress,location.address,location.postalCode,location.crossStreet
0,4dd91c4a8877f115101200b3,OFFICE,"[{'id': '4bf58dd8d48988d107951735', 'name': 'S...",v-1610650396,False,50.720121,-1.878498,"[{'label': 'display', 'lat': 50.72012091314346...",79,GB,Bournemouth,Dorset,United Kingdom,"[Bournemouth, Dorset]",NaN,NaN,NaN
1,4e2aa66162e144b5d3a35e5f,Yellow Buses - Enquiry Office,"[{'id': '4bf58dd8d48988d1fe931735', 'name': 'B...",v-1610650396,False,50.720819,-1.881771,"[{'label': 'display', 'lat': 50.720819, 'lng':...",306,GB,Bournemouth,Dorset,United Kingdom,"[Bournemouth, Dorset]",NaN,NaN,NaN
2,4d8a2b274757721e7d50b169,Lansdowne Sub Post Office,"[{'id': '4bf58dd8d48988d172941735', 'name': 'P...",v-1610650396,False,50.722536,-1.865882,"[{'label': 'display', 'lat': 50.72253639856578...",850,GB,Bournemouth,Dorset,United Kingdom,"[4 Lansdowne Crescent, Bournemouth, Dorset, BH...",4 Lansdowne Crescent,BH1 1RX,NaN
3,4bc5de6e5935c9b6527aa6d2,Bournemouth Delivery Office,"[{'id': '4bf58dd8d48988d172941735', 'name': 'P...",v-1610650396,False,50.728197,-1.863293,"[{'label': 'display', 'lat': 50.72819714188863...",1322,GB,Bournemouth,Dorset,United Kingdom,"[4 Wellington Rd, Bournemouth, Dorset]",4 Wellington Rd,NaN,NaN
4,5d5ff20156973a0008a449a5,Post Office,"[{'id': '4bf58dd8d48988d172941735', 'name': 'P...",v-1610650396,False,50.720270,-1.878686,"[{'label': 'display', 'lat': 50.72027, 'lng': ...",87,GB,Bournemouth,Dorset,United Kingdom,"[Old Christchurch Rd, Bournemouth, Dorset, BH1...",Old Christchurch Rd,BH1 1DY,NaN


In [233]:
map_folium = folium.Map(location=[latitude, longitude], zoom_start=14)
# Big marker circle I would like to disable the hand cursor
folium.Circle([latitude, longitude], 
                radius,
                fill=True,
                fill_color='#f72585',
                fill_opacity=0.1,
                stroke=False).add_to(map_folium)


if len(df.index) > 0 :
    # All the other markers on the map
    for venue, lat, lon in zip(df['name'], df['location.lat'], df['location.lng']):
        label_str = '{}'.format(venue)
        label = folium.Popup(label_str, parse_html=True)
        folium.CircleMarker(
            [lat, lon],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7).add_to(map_folium)
else:
    print('Not enough data to plot markers on the map')

map_folium